# File Ingestion

## Setup

Using local config file (see [01_pipeline](./01_pipeline.ipynb))

In [1]:
# import os
# import datajoint as dj;
# # change to the upper level folder to detect dj_local_conf.json
# if os.path.basename(os.getcwd())=='notebooks': os.chdir('..')
# assert os.path.basename(os.getcwd())=='adamacs', ("Please move to the main directory")
# from adamacs.pipeline import subject, behavior, session, event, trial

In [2]:
import os
# change to the upper level folder to detect dj_local_conf.json
if os.path.basename(os.getcwd())=='notebooks': os.chdir('..')
assert os.path.basename(os.getcwd())=='adamacs', ("Please move to the main directory")
import datajoint as dj; dj.conn()
from adamacs.pipeline import subject, behavior, session, event, trial


[2023-05-22 16:59:14,131][INFO]: Connecting tobiasr@172.26.128.53:3306
[2023-05-22 16:59:14,178][INFO]: Connected tobiasr@172.26.128.53:3306


Manual entry

### BPod Path Setup

Your `dj.config` file should have a section for your BPod root directory under `custom`: `exp_root_data_dir`. This is a list of one or more paths where the ingestion tool will look for the relative paths it is given.

In [3]:
import datajoint as dj
from element_interface.utils import find_full_path
from adamacs.pipeline import session, event, trial
from adamacs.ingest.bpod import Bpodfile
from adamacs.paths import get_experiment_root_data_dir

bpod_path = "DB_WEZ-8701_2022-03-18_scan9FB2LN5C_sess9FB2LN5C/scan9FB2LN5C_WEZ-8701_StimArenaMaster_20220318_165447.mat"
# bpod_path = "TR_WEZ-8701_2023-01-13_scan9FG1THFD_sess9FG1TAXY/scan9FG1THFD_WEZ-8701_StimArenaMaster_20230113_172307.mat"

root_dirs = dj.config["custom"]["exp_root_data_dir"]
bpod_path_full = find_full_path(get_experiment_root_data_dir(),bpod_path)

print(f"Root: {root_dirs}\nFull: {bpod_path_full}")

Root: ['/datajoint-data/data/tobiasr']
Full: /datajoint-data/data/tobiasr/DB_WEZ-8701_2022-03-18_scan9FB2LN5C_sess9FB2LN5C/scan9FB2LN5C_WEZ-8701_StimArenaMaster_20220318_165447.mat


### Initial check of tables

In [ ]:
# from adamacs.pipeline import session, event, trial

# session.Session.delete()


# print('Sessions:', len(session.Session()))
# print('Trials  :', len(trial.Trial()))
# print('Events  :', len(event.Event()))

In [ ]:
event.EventType.delete()

In [ ]:
dj.Diagram(behavior)+dj.Diagram(event)+dj.Diagram(trial)

## Automated BPod ingestion

The function is designed ask for a confirmation before entered into the schema.

In [ ]:
# bpod_path = "DB_WEZ-8701_2022-03-18_scan9FB2LN5C_sess9FB2LN5C/scan9FB2LN5C_WEZ-8701_StimArenaMaster_20220318_165447.mat"
root_dirs = dj.config["custom"]["exp_root_data_dir"]
bpod_object = Bpodfile(bpod_path)
bpod_object.ingest()


In [ ]:
subject.Subject()

Check that insertion worked:

In [ ]:
trial.TrialEvent & 'trial_id=0'

In [ ]:
trial.

In [ ]:
event.EventType()

We can also interact with bpod objects. For example:

In [ ]:
bpod_object.trial(1).attributes

In [ ]:
bpod_object.trial_data

# Add Harp recording

In [6]:
from adamacs.pipeline import behavior, event, scan
event_recording = event.BehaviorRecording.fetch('KEY')[0]
behavior.HarpRecording()

session_id,scan_id,harp_device_id


In [19]:
scansi = "scan9FB2LN5C"
scan_key = (scan.Scan & f'scan_id = "{scansi}"').fetch('KEY')[0]

bpod_path_relative = (event.BehaviorRecording.File & scan_key).fetch1("filepath")
harp_paths = list(find_full_path(
    get_experiment_root_data_dir(), bpod_path_relative
).parent.glob("*harp*bin"))

DataJointError: fetch1 should only return one tuple. 0 tuples found

In [21]:
event.BehaviorRecording.File()

session_id,scan_id,filepath
sess9FHS7Y22,scan9FHS7Y22,/datajoint-data/data/tobiasr/NK_ROS-1485_2023-04-28_scan9FHS7Y22_sess9FHS7Y22/scan9FHS7Y22_NK_ROS-1485_2024.h5
sess9FHS7Y22,scan9FHS845A,/datajoint-data/data/tobiasr/NK_ROS-1485_2023-04-28_scan9FHS845A_sess9FHS7Y22/scan9FHS845A_NK_ROS-1485_2025.h5
sess9FI3KWP3,scan9FI3KWP3,/datajoint-data/data/tobiasr/RN_OPI-1681_2023-05-17_scan9FI3KWP3_sess9FI3KWP3/scan9FI3KWP3_RN_OPI-1681_2025.h5
sess9FI3KWP3,scan9FI3L5PZ,/datajoint-data/data/tobiasr/RN_OPI-1681_2023-05-17_scan9FI3L5PZ_sess9FI3KWP3/scan9FI3L5PZ_RN_OPI-1681_2026.h5
sess9FI3LVI8,scan9FI3LVI8,/datajoint-data/data/tobiasr/LE_ROS-1518_2023-05-17_scan9FI3LVI8_sess9FI3LVI8/scan9FI3LVI8_LE_ROS-1518_2030.h5
sess9FI3MC3X,scan9FI3MC3X,/datajoint-data/data/tobiasr/LE_ROS-1518_2023-05-17_scan9FI3MC3X_sess9FI3MC3X/scan9FI3MC3X_LE_ROS-1518_2034.h5


In [14]:
bpod_path_relative

'/datajoint-data/data/tobiasr/LE_ROS-1518_2023-05-17_scan9FI3LVI8_sess9FI3LVI8/scan9FI3LVI8_LE_ROS-1518_2030.h5'

In [9]:
bpod_path_relative 

'/datajoint-data/data/tobiasr/LE_ROS-1518_2023-05-17_scan9FI3LVI8_sess9FI3LVI8/scan9FI3LVI8_LE_ROS-1518_2030.h5'

In [ ]:
behavior.HarpRecording.populate()

In [ ]:
behavior.HarpRecording.Channel()